In [ ]:
import ntpath
import numpy as np
import pandas as pd
from collections import Counter

In [ ]:
df_1 = pd.read_csv("G:/LiverCancer/predict_result/1_result.csv")
df_2 = pd.read_csv("G:/LiverCancer/predict_result/2_result.csv")
df_3 = pd.read_csv("G:/LiverCancer/predict_result/3_result.csv")
df_4 = pd.read_csv("G:/LiverCancer/predict_result/4_result.csv")
df_5 = pd.read_csv("G:/LiverCancer/predict_result/5_result.csv")
df_6 = pd.read_csv("G:/LiverCancer/predict_result/6_result.csv")
df_7 = pd.read_csv("G:/LiverCancer/predict_result/7_result.csv")
df_8 = pd.read_csv("G:/LiverCancer/predict_result/8_result.csv")
df_9 = pd.read_csv("G:/LiverCancer/predict_result/9_result.csv")
df_10 = pd.read_csv("G:/LiverCancer/predict_result/10_result.csv")
df = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10])

nucleus_path = df.iloc[:, 0].values.reshape(-1, 1)
nucleus_prob = np.argmax(df.iloc[:, 1:6].values, axis=1).reshape(-1, 1)
nucleus_info = np.hstack((nucleus_path,nucleus_prob)).tolist()

for i in range(10):
    
    split_rule = i+1

    src_path = "G:/LiverCancer/resources/split_rule/" + str(split_rule) +"/"

    test_biopsy_samples = pd.read_csv(src_path + "holdout.csv")["file_path"].tolist()
   
    rater_a = []
    rater_b = []

    for index, test_biopsy_sample in enumerate(test_biopsy_samples):

        if "grade0" in test_biopsy_sample: rater_a.append(0)
        if "grade1" in test_biopsy_sample: rater_a.append(1)
        if "grade2" in test_biopsy_sample: rater_a.append(2)
        if "grade3" in test_biopsy_sample: rater_a.append(3)
        if "grade4" in test_biopsy_sample: rater_a.append(4)

        biopsy_name = ntpath.basename(test_biopsy_sample)

        nucleus_label = [ int(nucleus[1]) for nucleus in nucleus_info if biopsy_name in nucleus[0] ]

        counter = Counter(nucleus_label)

        print(test_biopsy_sample, ":", counter)

        rater_b.append(int(counter.most_common()[0][0]))
    
    temp, conf = quadratic_weighted_kappa(rater_a,rater_b)

In [ ]:
def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None): 
    
    rater_a = np.array(rater_a, dtype=int) 
    rater_b = np.array(rater_b, dtype=int) 
    
    assert(len(rater_a) == len(rater_b)) 
    
    if min_rating is None: min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None: max_rating = max(max(rater_a), max(rater_b))
        
    conf_mat = confusion_matrix(rater_a, rater_b, min_rating, max_rating)
    
    print(conf_mat)
    
    num_ratings = len(conf_mat) 
    num_scored_items = float(len(rater_a)) 
    
    hist_rater_a = histogram(rater_a, min_rating, max_rating) 
    hist_rater_b = histogram(rater_b, min_rating, max_rating) 
    
    numerator = 0.0 
    denominator = 0.0 
    
    for i in range(num_ratings): 
        
        for j in range(num_ratings): 
            
            expected_count = (hist_rater_a[i] * hist_rater_b[j] / num_scored_items) 
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0) 
            numerator += d * conf_mat[i][j] / num_scored_items 
            denominator += d * expected_count / num_scored_items 
            
    return 1.0 - numerator / denominator, conf_mat 

def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):

    num_ratings = int(max_rating - min_rating + 1)

    conf_mat = [[0 for i in range(num_ratings)] for j in range(num_ratings)]

    for a, b in zip(rater_a, rater_b):

        conf_mat[a - min_rating][b - min_rating] += 1

    return conf_mat

def histogram(ratings, min_rating=None, max_rating=None):

    num_ratings = int(max_rating - min_rating + 1)

    hist_ratings = [0 for x in range(num_ratings)]

    for r in ratings:

        hist_ratings[r - min_rating] += 1
    
    return hist_ratings